### Federated Learning using Neural Network  Algorithm with Stochastic Gradient Descent Optimizer

### Installing Torch

PyTorch is a Python package that provides two high-level features:
- Tensor computation (like NumPy) with strong GPU acceleration
- Deep neural networks built on a tape-based autograd system

You can reuse your favorite Python packages such as NumPy, SciPy, and Cython to extend PyTorch when needed.

In [3]:
!pip install torch

     |████████████████████████████████| 108.0 MB 25 kB/s  eta 0:00:01    |████████████▍                   | 41.9 MB 868 kB/s eta 0:01:17     |████████████▉                   | 43.2 MB 1.6 MB/s eta 0:00:41     |████████████████████████▏       | 81.5 MB 734 kB/s eta 0:00:37     |████████████████████████▎       | 82.0 MB 734 kB/s eta 0:00:36     |█████████████████████████████▍  | 99.2 MB 1.8 MB/s eta 0:00:05


### Installing Syft

PySyft is a Python library for secure and private Deep Learning. PySyft decouples private data from model training, using Federated Learning, Differential Privacy, and Encrypted Computation (like Multi-Party Computation (MPC) and Homomorphic Encryption (HE)) within the main Deep Learning frameworks like PyTorch and TensorFlow. 

In [31]:
!pip install syft

### Import some python library
- Import `torch.nn.functional` for Rectified Linear Unit (ReLU) from torch 
- Import `torch.optim` for Stochastic Gradient Descent Optimizer
- import `syft` for simulate federated learning

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import logging
import syft as sy

### Use Hook Pytorch
`Torch Hook`, for adding extra functionalities to support Federated Learning and other private AI tools.
This experiment simulated 8 federated members as Virtual Worker

In [4]:
# hook PyTorch to PySyft i.e. add extra functionalities to support Federated Learning
# and other private AI tools
hook = sy.TorchHook(torch) 

# we create two imaginary schools
worker1 = sy.VirtualWorker(hook, id="worker1")
worker2 = sy.VirtualWorker(hook, id="worker2")
worker3 = sy.VirtualWorker(hook, id="worker3")
worker4 = sy.VirtualWorker(hook, id="worker4")
worker5 = sy.VirtualWorker(hook, id="worker5")
worker6 = sy.VirtualWorker(hook, id="worker6")
worker7 = sy.VirtualWorker(hook, id="worker7")
worker8 = sy.VirtualWorker(hook, id="worker8") 

### Initial parameter
- Setting parameter of batch size, epochs, learning rate, cuda, device, number of workers and etc

In [2]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 8, 'pin_memory': True} if use_cuda else {}

### Prepare and Import multiple file CSV
- Import `pandas` for manipulating, accessing, query dataset
- Import `glob` for reading multiple file
This experiment used 10 CSV files which contain 21 features and labels (six classes), with the name `mHealth_subject*.csv`. 

The dataset recorded the health activity from sensors. Each file contains the samples (by rows) recorded for all sensors (by columns). The labels used to identify the activities are similar to the ones.
- Column 1: acceleration from the chest sensor (X axis)
- Column 2: acceleration from the chest sensor (Y axis)
- Column 3: acceleration from the chest sensor (Z axis)
- Column 4: acceleration from the left-ankle sensor (X axis)
- Column 5: acceleration from the left-ankle sensor (Y axis)
- Column 6: acceleration from the left-ankle sensor (Z axis)
- Column 7: gyro from the left-ankle sensor (X axis)
- Column 8: gyro from the left-ankle sensor (Y axis)
- Column 9: gyro from the left-ankle sensor (Z axis)
- Column 10: magnetometer from the left-ankle sensor (X axis)
- Column 11: magnetometer from the left-ankle sensor (Y axis)
- Column 12: magnetometer from the left-ankle sensor (Z axis)
- Column 13: acceleration from the right-lower-arm sensor (X axis)
- Column 14: acceleration from the right-lower-arm sensor (Y axis)
- Column 15: acceleration from the right-lower-arm sensor (Z axis)
- Column 16: gyro from the right-lower-arm sensor (X axis)
- Column 17: gyro from the right-lower-arm sensor (Y axis)
- Column 18: gyro from the right-lower-arm sensor (Z axis)
- Column 19: magnetometer from the right-lower-arm sensor (X axis)
- Column 20: magnetometer from the right-lower-arm sensor (Y axis)
- Column 21: magnetometer from the right-lower-arm sensor (Z axis)
- Column 22: Label (0 for the null class)

The preprocessed data has 6 labels of activity. The activity set is listed as follows:
- L1: Standing still (1 min) 
- L2: Sitting and relaxing (1 min) 
- L3: Lying down (1 min) 
- L4: Walking (1 min) 
- L5: Climbing stairs (1 min) 
- L6: Running (1 min) 

After all the samples were imported, the process continued to split the training data and testing data. Finally, the dataset all ready to be used by 8 workers in the next step, the following code `inputs.shape process` showed `torch.Size([114123, 21])`, it explains that the number of samples is 114.123 and the number of columns is 21.

In [9]:
import pandas as pd
import numpy as np
import glob
import csv

#path = "drive/MyColab/emotional_dataset/angry_txt/"
#allFiles = glob.glob(os.path.join(path,"*.wav.txt"))

for filename in glob.glob('data/*.csv'):
  #print (filename)
      with open(filename, newline='') as f_input:
            fulltrainset = pd.read_csv(f_input,sep=',')
            X=fulltrainset.iloc[:,0:21]
            X=X.astype(np.float32)
            inputs = torch.tensor(X.values)
            y=fulltrainset.iloc[:,21]
            labels = torch.tensor(y.values)

            #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

            # splitting training and test data
            pct_test = 0.2
            train_labels = labels[:-int(len(labels)*pct_test)]
            train_inputs = inputs[:-int(len(labels)*pct_test)]
            test_labels = labels[-int(len(labels)*pct_test):]
            test_inputs = inputs[-int(len(labels)*pct_test):]

inputs.shape

torch.Size([114123, 21])

### Divide 8 workers to receive workload of training data and testing data
This experiment used 8 virtual workers and each worker will receive 1/8 of training and testing data. `sy.BaseDataset` is used to read the split dataset and then send to each worker.

In [10]:
# threshold indexes for dataset split
train_idx = int(len(train_labels)/8)
test_idx = int(len(test_labels)/8)

# Sending toy datasets to virtual workers
worker1_train_dataset = sy.BaseDataset(train_inputs[:train_idx], 
                                  train_labels[:train_idx]).send(worker1)
worker2_train_dataset = sy.BaseDataset(train_inputs[train_idx:], 
                                    train_labels[train_idx:]).send(worker2)
worker3_train_dataset = sy.BaseDataset(train_inputs[train_idx:], 
                                    train_labels[train_idx:]).send(worker3)
worker4_train_dataset = sy.BaseDataset(train_inputs[train_idx:], 
                                    train_labels[train_idx:]).send(worker4)
worker5_train_dataset = sy.BaseDataset(train_inputs[train_idx:], 
                                    train_labels[train_idx:]).send(worker5)
worker6_train_dataset = sy.BaseDataset(train_inputs[train_idx:], 
                                    train_labels[train_idx:]).send(worker6)
worker7_train_dataset = sy.BaseDataset(train_inputs[train_idx:], 
                                    train_labels[train_idx:]).send(worker7)
worker8_train_dataset = sy.BaseDataset(train_inputs[train_idx:], 
                                    train_labels[train_idx:]).send(worker8)
worker1_test_dataset = sy.BaseDataset(test_inputs[:test_idx], 
                                  test_labels[:test_idx]).send(worker1)
worker2_test_dataset = sy.BaseDataset(test_inputs[test_idx:], 
                                  test_labels[test_idx:]).send(worker2)
worker3_test_dataset = sy.BaseDataset(test_inputs[test_idx:], 
                                  test_labels[test_idx:]).send(worker3)
worker4_test_dataset = sy.BaseDataset(test_inputs[test_idx:], 
                                  test_labels[test_idx:]).send(worker4)
worker5_test_dataset = sy.BaseDataset(test_inputs[test_idx:], 
                                  test_labels[test_idx:]).send(worker5)
worker6_test_dataset = sy.BaseDataset(test_inputs[test_idx:], 
                                  test_labels[test_idx:]).send(worker6)
worker7_test_dataset = sy.BaseDataset(test_inputs[test_idx:], 
                                  test_labels[test_idx:]).send(worker7)
worker8_test_dataset = sy.BaseDataset(test_inputs[test_idx:], 
                                  test_labels[test_idx:]).send(worker8)

### Create federated dataset and data loader
`sy.FederatedDataset` is used to create federated Dataset which it is extension of Pytorch dataset class, while `sy.FederatedDataLoader` is used to cerate federated dataloaders, which it is extension of pytorch dataloader class

In [11]:
# Creating federated datasets, an extension of Pytorch TensorDataset class
federated_train_dataset = sy.FederatedDataset([worker1_train_dataset, worker2_train_dataset,worker3_train_dataset,
                                               worker4_train_dataset,worker5_train_dataset, worker6_train_dataset,
                                               worker7_train_dataset, worker8_train_dataset
                                              ])
federated_test_dataset = sy.FederatedDataset([worker1_test_dataset, worker2_test_dataset,
                                             worker3_test_dataset, worker4_test_dataset,
                                             worker5_test_dataset, worker6_test_dataset,
                                             worker7_test_dataset, worker8_test_dataset
                                             ])

# Creating federated dataloaders, an extension of Pytorch DataLoader class
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, 
                                                batch_size=args.batch_size, shuffle=True, **kwargs)
federated_test_loader = sy.FederatedDataLoader(federated_test_dataset, 
                                               batch_size=args.batch_size, shuffle=True, **kwargs)

#### Create Neural Network Model with Stochastic Gradient Descent Optimizer
This experiment used Neural Network Model with Stochastic Gradient Descent Optimizer, which it need to set up the input, hidden and output layer. The following code `class torch.nn.Linear(in_features: int, out_features: int, bias: bool = True)`, explain that the neural network has the number of feature input and out features. 

In the forward process, `x.view(-1, 21).squeeze(0)` explain that Neural Network receive the 2D input in pytorch format

To implement Stochastic Gradient Descent Optimizer in Neural Network, following code `optim.SGD(model.parameters(), lr=0.01)`

In [12]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(21, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 21).squeeze(0)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Model()
optimizer = optim.SGD(model.parameters(), lr=0.01)

### Building Model of mHealth Dataset

Finally, mHealth dataset can be built in the predictive model using 8 Virtual Workers and also Neural Network Model with Stochastic Gradient Descent Optimizer

This experiment used `epcoch`=5, and evaluated the model using `loss` parameter

In [13]:
for epoch in range(0, 5):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        # send the model to the client device where the data is present
        model.send(data.location)
        # training the model
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # get back the improved model
        model.get()
        if batch_idx % 100 == 0:
            # get back the loss
            loss = loss.get()
            print('Epoch: {:2d} [{:5d}/{:5d} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                batch_idx * 64,
                len(federated_train_loader) * 64,
                100. * batch_idx / len(federated_train_loader),
                loss.item()))
            


Epoch:  1 [    0/570624 (  0%)]	Loss: 2.874954
Epoch:  1 [ 6400/570624 (  1%)]	Loss: 0.377007
Epoch:  1 [12800/570624 (  2%)]	Loss: 0.787154
Epoch:  1 [19200/570624 (  3%)]	Loss: 0.357691
Epoch:  1 [25600/570624 (  4%)]	Loss: 0.385611
Epoch:  1 [32000/570624 (  6%)]	Loss: 0.317974
Epoch:  1 [38400/570624 (  7%)]	Loss: 0.440708
Epoch:  1 [44800/570624 (  8%)]	Loss: 0.389026
Epoch:  1 [51200/570624 (  9%)]	Loss: 0.481003
Epoch:  1 [57600/570624 ( 10%)]	Loss: 0.302120
Epoch:  1 [64000/570624 ( 11%)]	Loss: 0.500075
Epoch:  1 [70400/570624 ( 12%)]	Loss: 0.454674
Epoch:  1 [76800/570624 ( 13%)]	Loss: 0.362509
Epoch:  1 [83200/570624 ( 15%)]	Loss: 0.177408
Epoch:  1 [89600/570624 ( 16%)]	Loss: 0.314166
Epoch:  1 [96000/570624 ( 17%)]	Loss: 0.257121
Epoch:  1 [102400/570624 ( 18%)]	Loss: 0.395584
Epoch:  1 [108800/570624 ( 19%)]	Loss: 0.385994
Epoch:  1 [115200/570624 ( 20%)]	Loss: 0.285283
Epoch:  1 [121600/570624 ( 21%)]	Loss: 0.346868
Epoch:  1 [128000/570624 ( 22%)]	Loss: 0.189383
Epoch:  

### References

- https://blog.openmined.org/federated-learning-additive-secret-sharing-pysyft/
- https://www.simonwenkel.com/2019/07/20/introduction-to-pysyft.html
- https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines/